In [2]:
!pip install gradio hopsworks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.3/640.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# Load the model from Hopsworks
import hopsworks
import os
import joblib

with open('hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

mlp_model_meta = mr.get_model('mlp_flight_delay', version=1)
mlp_model_dir = mlp_model_meta.download()
mlp_model = joblib.load(f"{mlp_model_dir}/model.pkl")

Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1171606


In [50]:
import pandas as pd
import numpy as np
from plotly.graph_objs import Figure, Indicator

# Read data from Hopsworks feature store
feature_group = fs.get_feature_group(name="new_flight_data", version=1)
df_lagged = feature_group.read()

# Infer min and max values for sliders
slider_ranges = {}
continuous_features = [
    'arr_flights', 'arr_diverted', 'temperature_2m', 'wind_speed_10m',
    'arr_delay_lag1', 'arr_flights_lag2', 'weather_delay', 'security_delay',
    'carrier_delay', 'arr_delay_lag1_lag2', 'arr_delay_lag1_lag3'
]

for col in continuous_features:
    slider_ranges[col] = {
        "min": round(df_lagged[col].min(), 1),
        "max": round(df_lagged[col].max(), 1)
    }

carrier_mapping = {
    "American Airlines": "carrier_aa",
    "Delta Air Lines": "carrier_dl",
    "United Airlines": "carrier_ua",
    "Southwest Airlines": "carrier_wn",
    "Alaska Airlines": "carrier_as",
    "JetBlue Airways": "carrier_b6",
    "Spirit Airlines": "carrier_nk",
    "Frontier Airlines": "carrier_f9",
    "Envoy Air": "carrier_mq",
    "SkyWest Airlines": "carrier_oo",
    "Mesa Airlines": "carrier_oh",
    "Republic Airways": "carrier_yx",
    "Horizon Air": "carrier_qx",
    "Allegiant Air": "carrier_ax",
    "US Airways": "carrier_us",
    "AirTran Airways": "carrier_fl"
}

def encode_month(month):
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def predict_delay(carrier, arr_flights, arr_diverted, temperature, wind_speed, month,
                  arr_delay_lag1, arr_flights_lag2, weather_delay, security_delay,
                  carrier_delay, arr_delay_lag1_lag2, arr_delay_lag1_lag3):
    # Encode month for cyclical features
    month_sin, month_cos = encode_month(month)

    # Create input DataFrame
    data = {
        'arr_flights': [arr_flights],
        'arr_diverted': [arr_diverted],
        'temperature_2m': [temperature],
        'wind_speed_10m': [wind_speed],
        'month_sin': [month_sin],
        'month_cos': [month_cos],
        'arr_delay_lag1': [arr_delay_lag1],
        'arr_flights_lag2': [arr_flights_lag2],
        'weather_delay': [weather_delay],
        'security_delay': [security_delay],
        'carrier_delay': [carrier_delay],
        'arr_delay_lag1_lag2': [arr_delay_lag1_lag2],
        'arr_delay_lag1_lag3': [arr_delay_lag1_lag3]
    }
    input_df = pd.DataFrame(data)

    # One-hot encode carrier
    carrier_code = carrier_mapping.get(carrier)
    if carrier_code:
        input_df[carrier_code] = 1

    # Ensure all carrier columns exist and fill missing ones with 0
    for col in carrier_mapping.values():
        if col not in input_df:
            input_df[col] = 0

    # Reorder columns to match model's expected input
    input_df = input_df.reindex(columns=mlp_model.feature_names_in_, fill_value=0)

    # Make prediction
    prediction = mlp_model.predict(input_df)
    return round(prediction[0], 2)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.96s) 


In [72]:
import gradio as gr
import numpy as np
import plotly.graph_objects as go

# Define the prediction function for Gradio with Glowing Sphere and futuristic font
def predict_delay_interface(carrier, arr_flights, arr_diverted, temperature, wind_speed, month,
                            arr_delay_lag1, arr_flights_lag2, weather_delay, security_delay,
                            carrier_delay, arr_delay_lag1_lag2, arr_delay_lag1_lag3):
    # Get prediction
    delay_minutes = predict_delay(
        carrier, arr_flights, arr_diverted, temperature, wind_speed, month,
        arr_delay_lag1, arr_flights_lag2, weather_delay, security_delay,
        carrier_delay, arr_delay_lag1_lag2, arr_delay_lag1_lag3
    )

    # Create glowing sphere visualization
    sphere_radius = 1
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = sphere_radius * np.outer(np.cos(u), np.sin(v))
    y = sphere_radius * np.outer(np.sin(u), np.sin(v))
    z = sphere_radius * np.outer(np.ones(np.size(u)), np.cos(v))

    # Color mapping for delay (direct blue-to-red transition)
    color_scale = [
        [0.0, "blue"],  # Low delay
        [1.0, "red"]    # High delay
    ]
    normalized_delay = (delay_minutes + 1) / 6  # Normalize delay to 0-1 range (based on -1 to 5)

    # Plot sphere with glowing effect
    fig = go.Figure(data=[
        go.Surface(
            x=x, y=y, z=z,
            surfacecolor=np.full_like(x, normalized_delay),
            colorscale=color_scale,
            cmin=0, cmax=1,
            showscale=False,  # Hide colorbar for simplicity
            lighting=dict(ambient=0.5, diffuse=1, specular=1, roughness=0.3, fresnel=0.2),
            lightposition=dict(x=100, y=100, z=50)
        )
    ])

    # Add a 3D annotation for the delay value outside the sphere
    fig.add_trace(
        go.Scatter3d(
            x=[0], y=[1.5], z=[0],  # Place it above the sphere
            mode='text',
            text=[f"{round(delay_minutes, 2)}"],
            textfont=dict(
                size=30,  # Font size
                color="white",
                family="Orbitron, sans-serif"  # Futuristic sci-fi font
            ),
            showlegend=False
        )
    )

    fig.update_layout(
        title="How bad is the delay?",
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        margin=dict(l=0, r=0, t=10, b=0),
        paper_bgcolor="black",
    )

    # Add a 2D annotation for the delay value below the sphere
    fig.add_annotation(
        text=f"<b style='font-family:Orbitron, sans-serif; font-size:24px; color:white;'>{round(delay_minutes, 2)} minutes</b>",
        xref="paper", yref="paper",  # Reference the 2D plot space
        x=0.5, y=-0.1,  # Centered below the sphere (adjust `y` for distance)
        showarrow=False,
        align="center"
    )


    return fig

# Define input components
inputs = [
    gr.Dropdown(choices=list(carrier_mapping.keys()), label="Carrier"),
    gr.Slider(minimum=slider_ranges['arr_flights']['min'], maximum=slider_ranges['arr_flights']['max'], step=0.1, label="Arriving Flights Previous Month (Scaled)"),
    gr.Slider(minimum=slider_ranges['arr_diverted']['min'], maximum=slider_ranges['arr_diverted']['max'], step=0.1, label="Diverted Flights Previous Month (Scaled)"),
    gr.Slider(minimum=slider_ranges['temperature_2m']['min'], maximum=slider_ranges['temperature_2m']['max'], step=0.1, label="Temperature (Scaled)"),
    gr.Slider(minimum=slider_ranges['wind_speed_10m']['min'], maximum=slider_ranges['wind_speed_10m']['max'], step=0.1, label="Wind Speed (Scaled)"),
    gr.Slider(minimum=1, maximum=12, step=1, label="Month"),
    gr.Slider(minimum=slider_ranges['arr_delay_lag1']['min'], maximum=slider_ranges['arr_delay_lag1']['max'], step=0.1, label="Arrival Delay Previous Month (Scaled)"),
    gr.Slider(minimum=slider_ranges['arr_flights_lag2']['min'], maximum=slider_ranges['arr_flights_lag2']['max'], step=0.1, label="Arriving Flights 2 Months Ago (Scaled)"),
    gr.Slider(minimum=slider_ranges['weather_delay']['min'], maximum=slider_ranges['weather_delay']['max'], step=0.1, label="Weather Delay (Scaled)"),
    gr.Slider(minimum=slider_ranges['security_delay']['min'], maximum=slider_ranges['security_delay']['max'], step=0.1, label="Security Delay (Scaled)"),
    gr.Slider(minimum=slider_ranges['carrier_delay']['min'], maximum=slider_ranges['carrier_delay']['max'], step=0.1, label="Carrier Delay (Scaled)"),
    gr.Slider(minimum=slider_ranges['arr_delay_lag1_lag2']['min'], maximum=slider_ranges['arr_delay_lag1_lag2']['max'], step=0.1, label="Arrival Delay 2 Months Ago (Scaled)"),
    gr.Slider(minimum=slider_ranges['arr_delay_lag1_lag3']['min'], maximum=slider_ranges['arr_delay_lag1_lag3']['max'], step=0.1, label="Arrival Delay 3 Months Ago (Scaled)")
]

outputs = gr.Plot(label="Weather Prediction Visualization")

gr.Interface(
    fn=predict_delay_interface,
    inputs=inputs,
    outputs=outputs,
    title="Flight Delay Prediction with Glowing Sphere",
    description="Predict flight delays and visualize the delay as a glowing sphere that changes color. The delay value is displayed dynamically outside the sphere in a sci-fi font. Blue colors means low estimated delay and red colors mean a high estimated delay. The scaled delay time varies between circa -1 and 5."
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6db23d0208f9166549.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
